In [ ]:
import pandas as pd
import numpy as np

In [7]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


# Version 1

In [8]:
# ==============================
# 1. Load the data from Excel
# ==============================
file_path = "C:\\Users\\govar\\OneDrive\\Documents\\Data_Science_Journey\\data\\consultancy_data_5400.xlsx"
df = pd.read_excel(file_path)

# For consistency, let’s ensure key text columns are stripped and lower‐cased where needed.
df["level"] = df["level"].str.strip().str.lower()  # expecting values like "entry", "mid", "senior"
df["primary_specialization"] = df["primary_specialization"].fillna("").astype(str)

# ==============================
# 2. Compute Regional Averages
# ==============================
# We will use the average base_salary per region as a reference for cost-of-living fit.
region_avg_salary = df.groupby("region")["base_salary"].mean().to_dict()

# ==============================
# 3. Define City Profiles per Region
# ==============================
# For each region, we define three candidate cities.
# Each city has:
#   - cost_index (0 to 1): a proxy for cost-of-living (higher means more expensive)
#   - connectivity (0 to 1): a proxy for international connectivity (affects travel fit)
#   - favored_specializations: a list of keywords that we expect in the employee’s primary_specialization.
city_options = {
    "Americas": {
        "New York": {
            "cost_index": 0.9,
            "connectivity": 1.0,
            "favored_specializations": ["enterprise software", "cloud native", "devops", "strategic planning", "data architecture", "digital strategy"]
        },
        "San Francisco": {
            "cost_index": 0.95,
            "connectivity": 0.9,
            "favored_specializations": ["cloud native", "mobile development", "devops", "ui", "ux", "security architecture"]
        },
        "Toronto": {
            "cost_index": 0.7,
            "connectivity": 0.8,
            "favored_specializations": ["product strategy", "agile delivery", "digital transformation", "data architecture"]
        }
    },
    "EMEA": {
        "London": {
            "cost_index": 0.9,
            "connectivity": 1.0,
            "favored_specializations": ["design thinking", "digital transformation", "service design", "security architecture", "ux research", "product strategy"]
        },
        "Berlin": {
            "cost_index": 0.7,
            "connectivity": 0.8,
            "favored_specializations": ["security architecture", "mobile development", "cloud infrastructure", "devops", "data architecture"]
        },
        "Paris": {
            "cost_index": 0.85,
            "connectivity": 0.9,
            "favored_specializations": ["design", "service design", "product discovery", "creative"]
        }
    },
    "APAC": {
        "Singapore": {
            "cost_index": 0.9,
            "connectivity": 1.0,
            "favored_specializations": ["agile delivery", "portfolio management", "cloud native", "mobile development", "digital strategy"]
        },
        "Tokyo": {
            "cost_index": 0.9,
            "connectivity": 0.9,
            "favored_specializations": ["tech innovation", "devops", "security architecture", "cloud infrastructure"]
        },
        "Sydney": {
            "cost_index": 0.8,
            "connectivity": 0.8,
            "favored_specializations": ["product strategy", "digital transformation", "user research", "agile delivery"]
        }
    }
}

# ==============================
# 4. Set Up a Running Distribution Tracker
# ==============================
# For each region and candidate city, we will track counts of assigned employees per level,
# as well as a total count.
# (This lets our assignment function “learn” from previous assignments.)
city_distribution = {}
for region, cities in city_options.items():
    city_distribution[region] = {}
    for city in cities.keys():
        city_distribution[region][city] = {"entry": 0, "mid": 0, "senior": 0, "total": 0}

# ==============================
# 5. Define Ideal Level Distribution
# ==============================
# For example, we might want (roughly) 50% entry, 35% mid, 15% senior in a given city.
ideal_ratio = {"entry": 0.5, "mid": 0.35, "senior": 0.15}

# ==============================
# 6. Define Weights for the Composite Score
# ==============================
# These weights control the influence of each factor.
weight_spec   = 0.4  # specialization match
weight_level  = 0.3  # current level balance in the city
weight_salary = 0.2  # fit between employee’s salary and city cost index
weight_travel = 0.1  # travel/connectivity factor

# ==============================
# 7. Define the Assignment Function
# ==============================
def assign_city(row):
    region = row["region"]
    level = row["level"]  # already in lower-case: "entry", "mid", or "senior"
    base_salary = row["base_salary"]
    travel_pct = row["travel_percentage"]
    primary_spec = row["primary_specialization"].lower()  # for matching keywords
    
    # Retrieve the average salary for the region (for salary fit calculation)
    regional_avg = region_avg_salary.get(region, base_salary)
    
    best_city = None
    best_score = -np.inf  # start with a very low score
    
    # Loop over candidate cities for this region
    for city_name, attrs in city_options[region].items():
        # ----- Specialization Match -----
        # We check if any favored keyword appears in the employee's primary_specialization.
        spec_match = 0
        for keyword in attrs["favored_specializations"]:
            if keyword in primary_spec:
                spec_match = 1
                break  # found a match; no need to check further

        # ----- Level Balance Factor -----
        # We want to reward cities that currently have fewer employees of this level than the ideal.
        dist = city_distribution[region][city_name]
        total = dist["total"]
        current_level_count = dist[level]
        # If we were to add this employee, the desired count for that level is:
        desired_count = ideal_ratio[level] * (total + 1)
        # Compute a raw score as the difference normalized by the new total.
        raw_level_score = (desired_count - current_level_count) / (total + 1)
        # Normalize to a 0-to-1 scale.
        # If the city were perfectly balanced, current_count would equal desired_count and raw score = 0.
        # To shift so that 0.5 is “balanced”, we add ideal_ratio[level] and divide by 2*ideal_ratio[level].
        normalized_level_score = (raw_level_score + ideal_ratio[level]) / (2 * ideal_ratio[level])
        # (If the city is underrepresented for this level, normalized_level_score > 0.5; if overrepresented, < 0.5.)
        
        # ----- Salary Fit Factor -----
        # We compute a “salary ratio” and see how close it is to the city’s cost index.
        # (Here, a lower difference is better.)
        salary_ratio = base_salary / regional_avg  # e.g. 1.0 means at the average
        salary_fit_score = max(0, 1 - abs(salary_ratio - attrs["cost_index"]))
        
        # ----- Travel/Connectivity Factor -----
        # Higher travel percentage and higher city connectivity yield a better score.
        travel_fit_score = (travel_pct / 100) * attrs["connectivity"]
        
        # ----- Composite Score -----
        composite_score = (weight_spec * spec_match +
                           weight_level * normalized_level_score +
                           weight_salary * salary_fit_score +
                           weight_travel * travel_fit_score)
        
        # For debugging you might uncomment the next line:
        # print(f"Row {row['employee_id']} - {city_name}: spec={spec_match}, level={normalized_level_score:.2f}, salary={salary_fit_score:.2f}, travel={travel_fit_score:.2f} => score={composite_score:.2f}")
        
        if composite_score > best_score:
            best_score = composite_score
            best_city = city_name
            
    # Update the distribution counts for the chosen city
    city_distribution[region][best_city][level] += 1
    city_distribution[region][best_city]["total"] += 1
    
    return best_city

# ==============================
# 8. Apply the Assignment Function to Each Employee
# ==============================
# This will create (or overwrite) the "city" column with the new assignment.
df["city"] = df.apply(assign_city, axis=1)

# ==============================
# 9. Save the Updated Dataset
# ==============================
output_path = "C:\\Users\\govar\\OneDrive\\Documents\\Data_Science_Journey\\data\\consultancy_data_5400.xlsx"
df.to_excel(output_path, index=False)
print(f"City assignment completed. Updated dataset saved to {output_path}")


City assignment completed. Updated dataset saved to C:\Users\govar\OneDrive\Documents\Data_Science_Journey\data\consultancy_data_5400.xlsx


# Version 2

In [9]:
# ==============================================================================
# 1. Load and Pre-process the Data
# ==============================================================================
file_path = "C:\\Users\\govar\\OneDrive\\Documents\\Data_Science_Journey\\data\\consultancy_data_5400.xlsx"  # adjust as needed
df = pd.read_excel(file_path)

# Standardize key text columns
df["level"] = df["level"].str.strip().str.lower()  # expected values: "entry", "mid", "senior"
df["primary_specialization"] = df["primary_specialization"].fillna("").astype(str)
df["industry_expertise"] = df["industry_expertise"].fillna("").astype(str)
df["country"] = df["country"].str.strip()

# ==============================================================================
# 2. Compute Regional Average Salary (for Salary-Fit Calculations)
# ==============================================================================
region_avg_salary = df.groupby("region")["base_salary"].mean().to_dict()

# ==============================================================================
# 3. Define Expanded City Profiles per Region
# ==============================================================================
# Each city profile includes:
#   - cost_index (0 to 1): a proxy for cost-of-living (higher means more expensive)
#   - connectivity (0 to 1): how well-connected the city is internationally
#   - favored_specializations: keywords indicating the city’s industry strengths
#   - country: to allow bonus matching with the employee’s country
city_options = {
    "Americas": {
        "New York": {
            "cost_index": 0.9,
            "connectivity": 1.0,
            "favored_specializations": [
                "enterprise software", "cloud native", "devops",
                "strategic planning", "data architecture", "digital strategy"
            ],
            "country": "USA"
        },
        "San Francisco": {
            "cost_index": 0.95,
            "connectivity": 0.9,
            "favored_specializations": [
                "cloud native", "mobile development", "devops",
                "ui", "ux", "security architecture"
            ],
            "country": "USA"
        },
        "Toronto": {
            "cost_index": 0.7,
            "connectivity": 0.8,
            "favored_specializations": [
                "product strategy", "agile delivery", "digital transformation", "data architecture"
            ],
            "country": "Canada"
        }
    },
    "EMEA": {
        "London": {
            "cost_index": 0.9,
            "connectivity": 1.0,
            "favored_specializations": [
                "design thinking", "digital transformation", "service design",
                "security architecture", "ux research", "product strategy"
            ],
            "country": "UK"
        },
        "Berlin": {
            "cost_index": 0.7,
            "connectivity": 0.8,
            "favored_specializations": [
                "security architecture", "mobile development", "cloud infrastructure",
                "devops", "data architecture"
            ],
            "country": "Germany"
        },
        "Paris": {
            "cost_index": 0.85,
            "connectivity": 0.9,
            "favored_specializations": [
                "design", "service design", "product discovery", "creative"
            ],
            "country": "France"
        }
    },
    "APAC": {
        "Singapore": {
            "cost_index": 0.9,
            "connectivity": 1.0,
            "favored_specializations": [
                "agile delivery", "portfolio management", "cloud native",
                "mobile development", "digital strategy"
            ],
            "country": "Singapore"
        },
        "Tokyo": {
            "cost_index": 0.9,
            "connectivity": 0.9,
            "favored_specializations": [
                "tech innovation", "devops", "security architecture", "cloud infrastructure"
            ],
            "country": "Japan"
        },
        "Sydney": {
            "cost_index": 0.8,
            "connectivity": 0.8,
            "favored_specializations": [
                "product strategy", "digital transformation", "user research", "agile delivery"
            ],
            "country": "Australia"
        }
    }
}

# ==============================================================================
# 4. Initialize a Distribution Tracker for City Assignments
# ==============================================================================
# This tracker will help ensure that each city’s mix of employee levels remains
# roughly in line with a desired (ideal) ratio.
city_distribution = {}
for region, cities in city_options.items():
    city_distribution[region] = {}
    for city in cities.keys():
        city_distribution[region][city] = {"entry": 0, "mid": 0, "senior": 0, "total": 0}

# ==============================================================================
# 5. Set an Ideal Level Distribution
# ==============================================================================
# For instance, we might desire ~50% entry, 35% mid, and 15% senior employees per city.
ideal_ratio = {"entry": 0.5, "mid": 0.35, "senior": 0.15}

# ==============================================================================
# 6. Define Weights for the Composite Score
# ==============================================================================
# These weights control the influence of each factor in our decision.
weight_spec       = 0.25  # primary specialization match
weight_industry   = 0.10  # industry expertise match
weight_level      = 0.20  # current level balance in the city
weight_salary     = 0.15  # fit between employee’s salary and city cost index
weight_travel     = 0.10  # travel/connectivity factor (adjusted for remote work)
weight_country    = 0.10  # bonus if the employee’s country matches the city’s country
# (The weights sum to 1.0; you can adjust as needed.)

# ==============================================================================
# 7. Define the Supreme City Assignment Function (v3)
# ==============================================================================
def assign_city(row):
    """
    For a given employee (row), compute a composite score for each candidate city
    in the employee’s region by combining multiple factors (specialization, industry,
    level balance, salary fit, travel connectivity, and country match). Then, assign
    the city with the highest composite score.
    """
    region = row["region"]
    level = row["level"]
    base_salary = row["base_salary"]
    travel_pct = row["travel_percentage"]
    remote_ratio = row["remote_work_ratio"]  # e.g., 25.0 means 25%
    primary_spec = row["primary_specialization"].lower()
    industry_expertise = row["industry_expertise"].lower()
    employee_country = row["country"]

    # Use the regional average salary for normalization
    regional_avg = region_avg_salary.get(region, base_salary)
    best_city = None
    best_score = -np.inf

    # Evaluate each candidate city in the employee's region
    for city_name, attrs in city_options[region].items():
        # --- Primary Specialization Match ---
        spec_match = 0
        for keyword in attrs["favored_specializations"]:
            if keyword in primary_spec:
                spec_match = 1
                break

        # --- Industry Expertise Match ---
        industry_match = 0
        for keyword in attrs["favored_specializations"]:
            if keyword in industry_expertise:
                industry_match = 1
                break

        # --- Level Balance Factor ---
        # Favor cities that are underrepresented for the employee's level.
        dist = city_distribution[region][city_name]
        total = dist["total"]
        current_level_count = dist[level]
        desired_count = ideal_ratio[level] * (total + 1)
        raw_level_score = (desired_count - current_level_count) / (total + 1)
        normalized_level_score = (raw_level_score + ideal_ratio[level]) / (2 * ideal_ratio[level])
        
        # --- Salary Fit Factor ---
        # Compare the employee's salary (normalized by the regional average) with the city's cost index.
        salary_ratio = base_salary / regional_avg
        salary_fit_score = max(0, 1 - abs(salary_ratio - attrs["cost_index"]))
        
        # --- Travel/Connectivity Factor ---
        # A higher travel percentage and city connectivity yield a better score, but the impact
        # is reduced if the employee works remotely a lot.
        remote_modifier = 1 - (remote_ratio / 100)
        travel_fit_score = (travel_pct / 100) * attrs["connectivity"] * remote_modifier
        
        # --- Country Match Bonus ---
        country_bonus = 1 if employee_country.lower() == attrs["country"].lower() else 0
        
        # --- Composite Score ---
        composite_score = (
            weight_spec * spec_match +
            weight_industry * industry_match +
            weight_level * normalized_level_score +
            weight_salary * salary_fit_score +
            weight_travel * travel_fit_score +
            weight_country * country_bonus
        )
        
        # (Optional: Uncomment the following line to see a breakdown of the scores per city.)
        # print(f"Emp {row['employee_id']} -> {city_name}: spec={spec_match}, ind={industry_match}, level={normalized_level_score:.2f}, salary={salary_fit_score:.2f}, travel={travel_fit_score:.2f}, country={country_bonus} => score={composite_score:.2f}")
        
        if composite_score > best_score:
            best_score = composite_score
            best_city = city_name

    # Update the city distribution tracker with the chosen city
    city_distribution[region][best_city][level] += 1
    city_distribution[region][best_city]["total"] += 1

    return best_city

# ==============================================================================
# 8. Apply the Supreme Assignment Function to Each Employee
# ==============================================================================
df["city"] = df.apply(assign_city, axis=1)

# ==============================================================================
# 9. Save the Updated Dataset to an Excel File
# ==============================================================================
output_path = "C:\\Users\\govar\\OneDrive\\Documents\\Data_Science_Journey\\data\\consultancy_data_5400.xlsx"
df.to_excel(output_path, index=False)
print(f"Supreme city assignment (v3) completed. Updated dataset saved to {output_path}")


Supreme city assignment (v3) completed. Updated dataset saved to C:\Users\govar\OneDrive\Documents\Data_Science_Journey\data\consultancy_data_5400.xlsx
